In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,ConfusionMatrixDisplay
import pybaseball
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [11]:
pybaseball.cache.enable()

In [12]:
id = pybaseball.playerid_lookup("sugano","tomoyuki")
id

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,sugano,tomoyuki,608372,None,suganto01,-1,2025.0,2025.0


In [13]:
data = pybaseball.statcast_pitcher("2025-01-01","2025-06-19",608372)

In [14]:
#Filter results for Tomoyuki sugano's last played game 
game_data = data[data["game_date"] == "2025-06-08"]
game_data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
80,FC,2025-06-08,87.9,-2.46,5.98,"Sugano, Tomoyuki",691016,608372,field_out,hit_into_play,...,1.0,2.19,-0.14,0.14,38.8,2.009734,12.343537,27.561069,48.348106,18.223625
81,FC,2025-06-08,89.9,-2.30,6.03,"Sugano, Tomoyuki",691016,608372,None,ball,...,1.0,1.87,0.37,-0.37,39.2,NaN,NaN,NaN,NaN,NaN
82,SI,2025-06-08,92.8,-2.25,5.99,"Sugano, Tomoyuki",667670,608372,walk,ball,...,1.0,1.49,0.97,0.97,40.3,NaN,NaN,NaN,NaN,NaN
83,SI,2025-06-08,92.0,-2.61,5.81,"Sugano, Tomoyuki",667670,608372,None,ball,...,1.0,1.71,1.18,1.18,34.2,NaN,NaN,NaN,NaN,NaN
84,SI,2025-06-08,91.1,-2.31,5.98,"Sugano, Tomoyuki",667670,608372,None,foul,...,1.0,1.85,1.21,1.21,40.8,-4.982432,10.369901,37.144798,44.017193,16.639929


In [15]:
#Get the amount of pitches by each type 
game_data = game_data.groupby(["pitch_type"]).size()

In [20]:
game_data.head(6)

pitch_type
CU     7
FC    11
FF    15
FS    19
SI    20
ST    15
dtype: int64

In [21]:
#Compute total pitches 
total_pitches = sum(game_data)
print(total_pitches)

87


In [22]:
#Compute probability distributions
dists = []
for num in game_data:
    dists.append(round(num/total_pitches,2))
print(dists)

[0.08, 0.13, 0.17, 0.22, 0.23, 0.17]
